In [1]:
import nltk
from nltk.tokenize import word_tokenize

text = "Natural language Processing is fascinating."
tokens = word_tokenize(text)
print(tokens)

['Natural', 'language', 'Processing', 'is', 'fascinating', '.']


In [2]:
from nltk.stem import PorterStemmer

Stemmer = PorterStemmer()
words = ["running","ran","runs"]
stems = [Stemmer.stem(word) for word in words]
print(stems)

['run', 'ran', 'run']


In [3]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

lematizer = WordNetLemmatizer()
words = ["running","ran","runs"]
lemmas = [lematizer.lemmatize(word, pos ='v') for word in words]
print(lemmas)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\mehta\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


['run', 'run', 'run']


In [4]:
from nltk.corpus import stopwords
nltk.download('stopwords')

stop_words = set(stopwords.words ('english'))
filtered_text = [word for word in tokens if word.lower() not in stop_words]
print(filtered_text)

['Natural', 'language', 'Processing', 'fascinating', '.']


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mehta\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


**Designing and training a simple chatbot**

In [5]:
import pandas as pd

#load the dataset
data = pd.read_csv('chatbot_dataset.csv')

#preprocess the data
data['Question'] = data['Question'].apply(lambda x: ' '.join(nltk.word_tokenize(x.lower())))
print(data.head())

                                      Question  \
0                   introduction to the course   
1  overview of data science and its importance   
2    introduction to the data science workflow   
3         key skills and tools in data science   
4          where can i find my course videos ?   

                                              Answer  
0  Welcome to the data science course. Here you w...  
1  Data science is crucial for making informed de...  
2  The data science workflow includes data collec...  
3  Important skills include programming, statisti...  
4  You can find all your course videos on the Cip...  


C:\Users\mehta\AppData\Local\Temp\ipykernel_8404\2338154720.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


**Vectorizing text data**

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
x = vectorizer.fit_transform(data['Question'])
print(x.shape)

(48, 112)


**Training a Text Classification Model**

In [7]:
#Using Naive Bayes classifer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split

#split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(data['Question'], data['Answer'], test_size=0.2,random_state = 42)

# Create a model pipeline
model = make_pipeline(TfidfVectorizer(), MultinomialNB())

# Train the model
model.fit(X_train, y_train)
print("Model training complete.")

Model training complete.


# **Implementing a Function to Get Chatbot Responses**

In [8]:
# Function to get a response from the chatbot
def get_response(question):
    question = ' '.join(nltk.word_tokenize(question.lower()))
    answer = model.predict([question])
    return answer

# Testing the function
print(get_response("Can you explain Natural Language Processing?"))

['Datasets for the course can be downloaded from the resources section of each module on the Cipher Schools platform.']


## **Basics of Dash and Plotly**

**Initializing the Dash App**

In [9]:
import dash

# Initialize the Dash app
app = dash.Dash(__name__)

**Defining the Layout**

In [10]:
from dash import dcc, html

# Define the layout
app.layout = html.Div([
    html.H1("My Dash App", style={'textAlign': 'center'}),
    dcc.Input(id='input-box', type='text', value='Search here...'),
    html.Button('Submit', id='button'),
    html.Div(id='output-div')
])

# **Callback to Update the Output**

In [11]:
from dash.dependencies import Input, Output

    # Define callback to update output
@app.callback(
        Output('output-div', 'children'),
        Input('button', 'n_clicks'),
        [dash.dependencies.State('input-box', 'value')]
    )
def update_output(n_clicks, value):
        if n_clicks is not None:
            return f'You have entered: {value}'
        return ''


# **Adding a Text Area**

In [12]:
app.layout = html.Div([
    html.H1("Chatbot", style={'textAlign': 'center'}),
    dcc.Textarea(
        id='user-input',
        value='Type your question here...',
        style={'width': '100%', 'height': 100}
    ),
    html.Button('Submit', id='submit-button', n_clicks=0),
    html.Div(id='chatbot-output', style={'padding': '10px'})
])

**Creating a Chatbot Response**

In [13]:
# Define callback to update chatbot response
@app.callback(
    Output('chatbot-output', 'children'),
    Input('submit-button', 'n_clicks'),
    [dash.dependencies.State('user-input', 'value')]
)
def update_output(n_clicks, user_input):
    if n_clicks > 0:
        return html.Div([
            html.P(f"You: {user_input}", style={'margin': '10px'}),
            html.P(f"Bot: I am training now, ask something else.", style={'margin': '10px', 'backgroundColor': '#f0f0f0', 'padding': '10px', 'borderRadius': '5px'})
        ])
    return "Ask me something!"


In [14]:
if __name__ == '__main__':
    app.run_server(debug=True)